해당 코드는 로컬 PC에서 작성 및 실행한 코드입니다.   
각 팀원이 동시에 여러 모델을 학습시키는 과정에서   
YOLO, Mobilenet, Inceoption 모델은 로컬 PC를 사용하게 되었습니다.   
   
사용한 파이썬 버전과 라이브러리 목록입니다.   
python : 3.10.0   
tensorflow : 2.10.0   
tensorflow-gpu : 2.10.0   

In [ ]:
# tensorflow gpu 사용하도록 설정
import os
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
import math

# 데이터 경로
train_dir = 'C:/Users/handt/Desktop/finalpjt data/preprocessed_crop/train'
validation_dir = 'C:/Users/handt/Desktop/finalpjt data/preprocessed_crop/val'

# 이미지 데이터 전처리
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 픽셀 값을 0-1 사이로 정규화
    rotation_range=40,  # 이미지를 랜덤으로 회전
    width_shift_range=0.3,  # 이미지를 수평으로 랜덤 이동
    height_shift_range=0.3,  # 이미지를 수직으로 랜덤 이동
    shear_range=0.3,  # 이미지를 랜덤으로 시어 변환
    zoom_range=0.3,  # 이미지를 랜덤으로 확대/축소
    horizontal_flip=True,  # 이미지를 수평으로 랜덤 플립
    vertical_flip=True,  # 이미지를 수직으로 랜덤 플립
    fill_mode='nearest')  # 변환 후 빈 공간을 채우는 방식

validation_datagen = ImageDataGenerator(rescale=1./255)  # 검증 데이터, 정규화만 사용

# 데이터 로드
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

# 클래스 이름 보여주기
print("클래스 이름 (train):", train_generator.class_indices)
print("클래스 이름 (validation):", validation_generator.class_indices)

# MobileNetV2 모델 로드
base_model = MobileNetV2(input_shape=(224, 224, 3),  # 입력 이미지 형태
                         include_top=False,  # 최상위 분류 층 제외
                         weights='imagenet')  # ImageNet 사전 학습 가중치 사용

# 모델 맞춤화
x = base_model.output  # 기본 모델의 출력
x = GlobalAveragePooling2D()(x)  # 전역 평균 풀링 층 추가
x = BatchNormalization()(x)  # 배치 정규화 층 추가
x = Dense(1024, activation='relu')(x)  # 완전 연결 층 추가
x = Dropout(0.5)(x)  # 드롭아웃 층 추가
x = Dense(512, activation='relu')(x)  # 추가 완전 연결 층
x = Dropout(0.5)(x)  # 드롭아웃 층 추가
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # 출력 층, 클래스 수 만큼 뉴런 설정

model = Model(inputs=base_model.input, outputs=predictions)  # 전체 모델 정의

# 일부 상위 레이어를 학습 가능하게 설정
base_model.trainable = True  # 기본 모델의 모든 층을 학습 가능하게 설정
fine_tune_at = 50  # 상위 50개 레이어를 제외한 나머지 레이어를 학습 가능하게 설정
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False  # 하위 50개 레이어는 학습 불가능하게 설정

# 학습률 스케줄링 함수 정의
def scheduler(epoch, lr):
    if epoch < 10:
        return lr  # 초기 10 에포크 동안 학습률 유지
    else:
        return lr * math.exp(-0.1)  # 이후 학습률 감소

callback = LearningRateScheduler(scheduler)  # 학습률 스케줄링 콜백

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=1e-4),  # Adam 옵티마이저와 학습률 설정
              loss='categorical_crossentropy',  # 손실 함수 설정
              metrics=['accuracy'])  # 평가 지표 설정

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # 에포크당 스텝 수
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # 검증 스텝 수
    callbacks=[callback])

# 모델 평가
model.evaluate(validation_generator)  # 검증 데이터로 모델 평가

model.save('saved_model_mobilenetv2_improved.h5')  # 모델 저장


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
import math

# 데이터 경로
train_dir = 'C:/Users/handt/Desktop/finalpjt data/preprocessed_crop/train'
validation_dir = 'C:/Users/handt/Desktop/finalpjt data/preprocessed_crop/val'

# 이미지 데이터 전처리
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 픽셀 값을 0-1 사이로 정규화
    rotation_range=40,  # 이미지를 랜덤으로 회전
    width_shift_range=0.3,  # 이미지를 수평으로 랜덤 이동
    height_shift_range=0.3,  # 이미지를 수직으로 랜덤 이동
    shear_range=0.3,  # 이미지를 랜덤으로 시어 변환
    zoom_range=0.3,  # 이미지를 랜덤으로 확대/축소
    horizontal_flip=True,  # 이미지를 수평으로 랜덤 플립
    vertical_flip=True,  # 이미지를 수직으로 랜덤 플립
    fill_mode='nearest')  # 변환 후 빈 공간을 채우는 방식

validation_datagen = ImageDataGenerator(rescale=1./255) # 검증 데이터, 정규화만 사용

# 데이터 로드
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

# 클래스 이름 출력
print("클래스 이름 (train):", train_generator.class_indices)
print("클래스 이름 (validation):", validation_generator.class_indices)

# MobileNetV3 모델 로드
base_model = MobileNetV3Large(input_shape=(224, 224, 3),
                              include_top=False,
                              weights='imagenet')

# 모델 맞춤화
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.6)(x)  # 드롭아웃 비율 증가
x = Dense(512, activation='relu')(x)
x = Dropout(0.6)(x)  # 드롭아웃 비율 증가
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# 일부 상위 레이어를 학습 가능하게 설정
base_model.trainable = True
fine_tune_at = 50
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 콜백 정의
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[reduce_lr])  # 변경된 콜백

# 모델 평가
model.evaluate(validation_generator)

model.save('saved_model_mobilenetv3_improved.h5')



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# 데이터 경로 설정
train_dir = 'C:/Users/handt/Desktop/finalpjt data/preprocessed_crop/train'
validation_dir = 'C:/Users/handt/Desktop/finalpjt data/preprocessed_crop/val'

# 이미지 데이터 전처리 설정 (학습 데이터용)
train_datagen = ImageDataGenerator(
    rescale=1./255,  # 픽셀 값을 0-1 사이로 정규화
    rotation_range=40,  # 이미지를 랜덤으로 회전
    width_shift_range=0.3,  # 이미지를 수평으로 랜덤 이동
    height_shift_range=0.3,  # 이미지를 수직으로 랜덤 이동
    shear_range=0.3,  # 이미지를 랜덤으로 시어 변환
    zoom_range=0.3,  # 이미지를 랜덤으로 확대/축소
    horizontal_flip=True,  # 이미지를 수평으로 랜덤 플립
    vertical_flip=True,  # 이미지를 수직으로 랜덤 플립
    fill_mode='nearest')  # 변환 후 빈 공간을 채우는 방식

# 이미지 데이터 전처리 설정 (검증 데이터용)
validation_datagen = ImageDataGenerator(rescale=1./255)  # 검증 데이터, 정규화만 사용

# 데이터 로드
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=16,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=16,
    class_mode='categorical')

# 클래스 이름 출력
print("클래스 이름 (train):", train_generator.class_indices)
print("클래스 이름 (validation):", validation_generator.class_indices)

# 인셉션V3 모델 로드 (사전 학습된 가중치 사용)
base_model = InceptionV3(input_shape=(299, 299, 3),
                         include_top=False,  # 최상위 레이어 제외
                         weights='imagenet')  # 이미지넷 가중치 사용

# 모델 맞춤화 (기본 모델 위에 커스텀 레이어 추가)
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 전역 평균 풀링 레이어
x = BatchNormalization()(x)  # 배치 정규화 레이어
x = Dense(1024, activation='relu')(x)  # 완전 연결 레이어 + ReLU 활성화 함수
x = Dropout(0.5)(x)  # 드롭아웃 레이어 (과적합 방지)
x = Dense(512, activation='relu')(x)  # 또 다른 완전 연결 레이어 + ReLU 활성화 함수
x = Dropout(0.5)(x)  # 드롭아웃 레이어 (과적합 방지)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # 출력 레이어 (다중 클래스 분류용 소프트맥스 활성화 함수)

# 최종 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 기본 모델의 일부 레이어를 고정 (학습 불가능하게 설정)
base_model.trainable = True
fine_tune_at = 172  # 특정 레이어부터 학습 가능하게 설정
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# 모델 컴파일 (손실 함수와 옵티마이저 설정)
model.compile(optimizer=Adam(learning_rate=1e-4),  # Adam 옵티마이저 + 학습률 설정
              loss='categorical_crossentropy',  # 손실 함수 설정 (다중 클래스 분류용)
              metrics=['accuracy'])  # 평가 지표 설정

# 학습률 감소 콜백 정의 (validation loss가 개선되지 않을 때 학습률 감소)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # 에포크 당 스텝 수
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # 검증 스텝 수
    callbacks=[reduce_lr])  # 콜백 설정

# 모델 평가 (검증 데이터)
model.evaluate(validation_generator)

# 모델 저장
model.save('saved_model_inceptionv3_improved.h5')
